<a href="https://colab.research.google.com/github/gastonviveros2411/labo2025v/blob/main/src/ensembles/Tarea_clase4_%20420_ArbolesAzarosos_final.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Ensembles de Arboles de Decision

Un arbol de decisión es un modelo débil, el aumento del poder predictivo proviene al ensamblar varios arboles de decisión.
<br> Si promedio n arboles identicos, el resultados es exactamente el mismo que utilizar un solo arbol, necesito PERTURBAR cada arbol para disponer de variablidad

la variabilidad provendrá de estas fuentes:


*   Perturbar el dataset
*   Perturbar el algoritmo del arbol
*   Perturbar el dataset y el algoritmo del arbol al mismo tiempo

Se verán estos tres algoritmos


*   Arboles Azarosos
*   Random Forest
*   Gradient Boosting of Decision Trees

#### 4.01 Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



---



## 4.02 Arboles Azarosos

Arboles Azarosos es el nombre de un algoritmo trivial (por favor NO confundir con Random Forest)
Qué tipo de perturbaciones se realizan en Arboles Azarosos
* Se perturba el dataset
* No se perturba el algoritmo, es siempre rpart original

Cada  arbolito de  Arboles Azarosos se entrena sobre un dataset perturbado,  que tiene exactamente la misma cantidad de registros pero solo un subconjunto de los atributos (campos)  del dataset, tomados al azar, de los originales.
<br> En esta primera corrida, se construira cada arbol en un dataset utilizando el 50% de los campos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Nov 03 02:14:52 PM 2025"

In [2]:
require("data.table")
require("rpart")

if (!require("rBayesianOptimization")) {
  install.packages("rBayesianOptimization", repos="http://cran.us.r-project.org")
  library("rBayesianOptimization")
}

Loading required package: data.table

Loading required package: rpart

Loading required package: rBayesianOptimization

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rBayesianOptimization’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘iterators’, ‘lhs’, ‘foreach’, ‘GPfit’




In [3]:
PARAM <- list()
PARAM$semilla_primigenia <- 102191  # ← CAMBIAR

PARAM$feature_fraction <- 0.5
PARAM$num_trees <- 32
PARAM$cp_fijo <- -1

PARAM$bo_iteraciones <- 50
PARAM$bo_init_points <- 10

cat("⚡ CONFIGURACIÓN RÁPIDA\n")
cat("Semilla:", PARAM$semilla_primigenia, "\n")
cat("cp (FIJO):", PARAM$cp_fijo, "\n")
cat("Evaluaciones:", PARAM$bo_iteraciones + PARAM$bo_init_points, "\n")
cat("Tiempo estimado: 1.5-2 horas\n")

⚡ CONFIGURACIÓN RÁPIDA
Semilla: 102191 
cp (FIJO): -1 
Evaluaciones: 60 
Tiempo estimado: 1.5-2 horas


In [4]:
setwd("/content/buckets/b1/exp")
experimento <- "exp4020_rapido"
dir.create(experimento, showWarnings=FALSE)
setwd(paste0("/content/buckets/b1/exp/", experimento))

In [5]:
dataset <- fread("/content/datasets/dataset_pequeno.csv")
dtrain <- dataset[foto_mes == 202107]

set.seed(PARAM$semilla_primigenia)
indices_train <- sample(1:nrow(dtrain), size=0.7*nrow(dtrain))
dtrain_real <- dtrain[indices_train]
dvalidation <- dtrain[-indices_train]

dfuture <- dataset[foto_mes == 202109]
dfuture[, clase_ternaria := NA]

campos_buenos <- copy(setdiff(colnames(dtrain_real), c("clase_ternaria")))

cat("Training:", nrow(dtrain_real), "\n")
cat("Validation:", nrow(dvalidation), "\n")
cat("Campos:", length(campos_buenos), "\n")

Training: 115217 
Validation: 49379 
Campos: 154 


In [6]:
calcular_ganancia <- function(probs, real, corte) {
  predicho <- as.numeric(probs > corte)
  ganancia <- sum(
    (predicho == 1 & real == "BAJA+2") * 117000 +
    (predicho == 1 & real != "BAJA+2") * (-3000)
  )
  return(ganancia)
}

entrenar_ensemble <- function(minsplit, minbucket, maxdepth) {
  cp <- PARAM$cp_fijo

  minsplit <- as.integer(round(minsplit))
  minbucket <- as.integer(round(minbucket))
  maxdepth <- as.integer(round(maxdepth))

  if (minbucket > minsplit / 2) minbucket <- as.integer(minsplit / 2)
  if (minsplit < 2) minsplit <- 2
  if (minbucket < 1) minbucket <- 1
  if (maxdepth < 1) maxdepth <- 1
  if (maxdepth > 30) maxdepth <- 30

  prob_acum_val <- numeric(nrow(dvalidation))

  for (arbol in 1:PARAM$num_trees) {
    qty_campos <- as.integer(length(campos_buenos) * PARAM$feature_fraction)
    campos_random <- sample(campos_buenos, qty_campos)
    campos_str <- paste(campos_random, collapse=" + ")
    formulita <- paste0("clase_ternaria ~ ", campos_str)

    tryCatch({
      modelo <- rpart(formulita, data=dtrain_real, xval=0,
        control=list(cp=cp, minsplit=minsplit, minbucket=minbucket, maxdepth=maxdepth))
      pred <- predict(modelo, dvalidation, type="prob")
      prob_acum_val <- prob_acum_val + pred[, "BAJA+2"]
    }, error = function(e) NULL)
  }

  prob_promedio <- prob_acum_val / PARAM$num_trees

  # ⚡ CLAVE: Solo 20 cortes
  cortes <- seq(0.025, 0.5, by=0.025)
  ganancias <- sapply(cortes, function(c) calcular_ganancia(prob_promedio, dvalidation$clase_ternaria, c))

  return(max(ganancias))
}

cat("✓ Funciones cargadas\n")

✓ Funciones cargadas


In [7]:
iteracion_global <- 0
mejor_ganancia_global <- -Inf
mejores_params_global <- list()

historial <- data.table(
  iteracion=integer(), cp=numeric(), minsplit=numeric(),
  minbucket=numeric(), maxdepth=numeric(), ganancia=numeric(), tiempo_seg=numeric()
)

funcion_objetivo <- function(minsplit, minbucket, maxdepth) {
  iteracion_global <<- iteracion_global + 1
  tiempo_inicio <- Sys.time()

  if (iteracion_global %% 5 == 0 || iteracion_global <= 3) {
    cat(sprintf("\n[%d] cp=%d ms=%d mb=%d md=%d",
      iteracion_global, PARAM$cp_fijo, round(minsplit), round(minbucket), round(maxdepth)))
  }

  ganancia <- entrenar_ensemble(minsplit, minbucket, maxdepth)

  tiempo_fin <- Sys.time()
  tiempo_transcurrido <- as.numeric(difftime(tiempo_fin, tiempo_inicio, units="secs"))

  if (iteracion_global %% 5 == 0 || iteracion_global <= 3) {
    cat(sprintf(" → $%.1fM (%.0fs)", ganancia/1e6, tiempo_transcurrido))
  }

  if (ganancia > mejor_ganancia_global) {
    mejor_ganancia_global <<- ganancia
    mejores_params_global <<- list(cp=PARAM$cp_fijo, minsplit=round(minsplit),
      minbucket=round(minbucket), maxdepth=round(maxdepth))
    cat(" ★")
  }

  historial <<- rbind(historial, data.table(iteracion=iteracion_global, cp=PARAM$cp_fijo,
    minsplit=round(minsplit), minbucket=round(minbucket), maxdepth=round(maxdepth),
    ganancia=ganancia, tiempo_seg=tiempo_transcurrido))

  if (iteracion_global %% 10 == 0) {
    fwrite(historial, "historial_bayesiano_parcial.csv")
    cat(sprintf(" [Mejor: $%.1fM]", mejor_ganancia_global/1e6))
  }

  return(list(Score = ganancia / 1e6))
}

cat("✓ Función objetivo lista\n")

✓ Función objetivo lista


In [8]:
bounds <- list(minsplit = c(10, 400), minbucket = c(5, 200), maxdepth = c(3, 15))

cat("Rangos:\n")
cat("  cp:", PARAM$cp_fijo, "(FIJO)\n")
cat("  minsplit:", bounds$minsplit, "\n")
cat("  minbucket:", bounds$minbucket, "\n")
cat("  maxdepth:", bounds$maxdepth, "\n")

Rangos:
  cp: -1 (FIJO)
  minsplit: 10 400 
  minbucket: 5 200 
  maxdepth: 3 15 


In [ ]:
cat("\n=== INICIO OPTIMIZACIÓN ===\n\n")

set.seed(PARAM$semilla_primigenia)
tiempo_inicio_total <- Sys.time()

resultado_bo <- BayesianOptimization(
  FUN = funcion_objetivo,
  bounds = bounds,
  init_points = PARAM$bo_init_points,
  n_iter = PARAM$bo_iteraciones,
  acq = "ucb",
  kappa = 2.576,
  eps = 0.0,
  verbose = FALSE
)

tiempo_fin_total <- Sys.time()
tiempo_total <- as.numeric(difftime(tiempo_fin_total, tiempo_inicio_total, units="mins"))

cat("\n\n=== COMPLETADO ===\n")
cat(sprintf("Tiempo: %.1f min (%.2fh)\n", tiempo_total, tiempo_total/60))


=== INICIO OPTIMIZACIÓN ===



In [ ]:
fwrite(historial, "historial_bayesiano_completo.csv")
historial_ordenado <- historial[order(-ganancia)]

cat("\nTOP 10:\n")
print(historial_ordenado[1:min(10, nrow(historial_ordenado))])

mejor_config <- historial_ordenado[1]

cat("\n🏆 MEJOR:\n")
cat("  cp:", mejor_config$cp, "(F)\n")
cat("  minsplit:", mejor_config$minsplit, "\n")
cat("  minbucket:", mejor_config$minbucket, "\n")
cat("  maxdepth:", mejor_config$maxdepth, "\n")
cat("  Ganancia: $", format(mejor_config$ganancia, big.mark=","), "\n")

In [ ]:
cat("\nModelo final...\n")

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob_acumulada := 0]

set.seed(PARAM$semilla_primigenia)

for (arbolito in seq(PARAM$num_trees)) {
  if (arbolito %% 8 == 0) cat(arbolito, " ")

  qty_campos <- as.integer(length(campos_buenos) * PARAM$feature_fraction)
  campos_random <- sample(campos_buenos, qty_campos)
  formulita <- paste0("clase_ternaria ~ ", paste(campos_random, collapse=" + "))

  modelo <- rpart(formulita, data=dtrain, xval=0,
    control=list(cp=mejor_config$cp, minsplit=mejor_config$minsplit,
      minbucket=mejor_config$minbucket, maxdepth=mejor_config$maxdepth))

  prediccion <- predict(modelo, dfuture, type="prob")
  tb_prediccion[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]
}

cat("\n")

cortes_test <- seq(0.5, 2.0, by=0.1) * PARAM$num_trees
envios <- sapply(cortes_test, function(c) sum(tb_prediccion$prob_acumulada > c))
idx_corte <- which.min(abs(envios - 10000))
umbral_corte <- cortes_test[idx_corte]

tb_prediccion[, Predicted := as.numeric(prob_acumulada > umbral_corte)]

archivo_kaggle <- "KA420_bayesiano_rapido_cp_fijo.csv"
fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)], file=archivo_kaggle, sep=",")

cat("Archivo:", archivo_kaggle, "\n")
cat("Predichos:", sum(tb_prediccion$Predicted), "\n")

In [ ]:
mensaje <- sprintf("BO-RAPIDA 32a cp=-1 ms=%d mb=%d md=%d",
  mejor_config$minsplit, mejor_config$minbucket, mejor_config$maxdepth)

linea <- paste(
  "kaggle competitions submit",
  "-c labo-i-2025-ba-analista-sr",
  "-f", archivo_kaggle,
  "-m '", mensaje, "'"
)

cat("\nSubiendo...\n")
salida <- system(linea, intern=TRUE)
cat(salida, sep="\n")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")